<a href="https://colab.research.google.com/github/byui-cse/cse380-notebooks/blob/master/09_2_Ponder_and_Prove_Data_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ponder and Prove Data Compression
#### Due: Saturday, 6 March 2021, 11:59 pm.

# TODO Explore Huffman Trees and Huffman Codes


Your task is examine how to compress a *special piece of information* as compactly as possible, and **calculate various compression ratios**.

Recall that the **compression ratio** of a variable-length encoding like Huffman encoding is the percentage $100(f - v)/f$, where $f$ is the number of bits per symbol of the smallest **fixed**-length encoding, and $v$ is the average number of bits per symbol with the variable-length encoding.

For example, if there were 9 different symbols in a message, $f=4$ is the number of bits of the smallest fixed-length encoding, because $2^3 = 8$ (not enough for $9$) and $2^4 = 16$ (enough and to spare). If the variable-length encoding of the message had $v=3.12$, the compression ratio would be $100(4 - 3.12)/4 \approx 22\%$.

Note that calculating the average number of bits per symbol is not strictly necessary. That's because an alternate and equivalent way is to calculate $100(ft - vt)/ft$, where $ft$ is the **total** number of bits encoded with the fixed encoding, and $vt$ is the **total** number of bits encoded with the variable-length encoding.

The *special piece of information* to be compressed is a list of the first ten million primes. This is a list that starts

|    |
|----|
|  2 |
|  3 |
|  5 |
|  7 |
| 11 |
| 13 |
| 17 |
| 19 |
| 23 |
| 29 |

  and ends

|           |
|-----------|
| 179424551 |
| 179424571 |
| 179424577 |
| 179424601 |
| 179424611 |
| 179424617 |
| 179424629 |
| 179424667 |
| 179424671 |
| 179424673 |

As ASCII text stored in a file with one prime per line, the size of this data file is slightly over 89 megabytes. The goal is to compress this down to just over 5 megabytes (5589056 bytes, to be exact). That's a 94% compression ratio!

Standard compression tools can only get about a 73% compression ratio for this ASCII data. A more clever approach is needed. Instead of compressing the list of prime numbers, compress a list of the *gaps* between them!

It doesn't save much, just the unique (occurring only once) gap size of 1 between 2 and 3, but in the spirit of de Polignac's conjecture that every *even* number appears infinitely often as a gap between consecutive primes, just consider the even-sized gaps. The result will be a list that starts with 2 (the difference between 5 and 3), 2 (the difference between 7 and 5), 4 (the difference between 11 and 7), 2 (the difference between 13 and 11), 4 (the difference between 17 and 13), 2 (the difference between 19 and 17), 4 (the difference between 23 and 19), and 6 (the difference between 29 and 23).

Generating this data is the first task. The algorithm for doing so is very straightforward:

1. Find the gaps between consecutive odd primes.
2. Store these gaps as a list of even numbers.

Tabulating the results, the first ten gaps and the last ten gaps are as follows, where the numbers after the equals signs are the gaps to list:

|                 |
|-----------------|
|  5  -   3  =  2 |
|  7  -   5  =  2 |
| 11  -   7  =  4 |
| 13  -  11  =  2 |
| 17  -  13  =  4 |
| 19  -  17  =  2 |
| 23  -  19  =  4 |
| 29  -  23  =  6 |
| 31  -  29  =  2 |
| 37  -  31  =  6 |

|                                |
|--------------------------------|
| 179424551  -  179424533  =  18 |
| 179424571  -  179424551  =  20 |
| 179424577  -  179424571  =   6 |
| 179424601  -  179424577  =  24 |
| 179424611  -  179424601  =  10 |
| 179424617  -  179424611  =   6 |
| 179424629  -  179424617  =  12 |
| 179424667  -  179424629  =  38 |
| 179424671  -  179424667  =   4 |
| 179424673  -  179424671  =   2 |

As a correctness check, see if your generated list of gaps has length 9999998.

The next step is to count how many times each gap size occurs, so that for the Huffman encoding scheme, the larger the frequency of occurrence, the smaller the number of bits encoding that gap size.

As a correctness check, here are the first ten and the last ten gap counts:

|  Gap | Count   |
|------|---------|
|    2 |  738597 |
|    4 |  738717 |
|    6 | 1297540 |
|    8 |  566151 |
|   10 |  729808 |
|   12 |  920661 |
|   14 |  503524 |
|   16 |  371677 |
|   18 |  667734 |
|   20 |  354267 |
|      |         |
|  190 |       1 |
|  192 |       3 |
|  194 |       1 |
|  196 |       1 |
|  198 |       6 |
|  202 |       2 |
|  204 |       3 |
|  210 |       4 |
|  220 |       1 |
|  222 |       1 |

Note two things from these partial gap counts:

1. Small even numbers (< 100) are well represented, larger ones (< 1000) less so.
2. Ten million primes aren't enough to have *every* even number represented; for example, 200, 206, 208, 212, 214, 216, and 218 do not appear even once.


In [17]:
from pyprimesieve import primes
import numpy as np
ten_millionth_prime = 179424673

first_ten_million_primes = list(primes(ten_millionth_prime+1))

def list_the_gaps(n):
    return list(map(lambda i: n[i]-n[i-1],range(2,len(n))))

gaps = list_the_gaps(first_ten_million_primes)

#numpy array of primes
np_gaps = np.array(gaps) 
# Count unique elements
unique, counts = np.unique(np_gaps, return_counts=True)
gap_counts = dict(zip(unique, counts))


print(f"First Ten million primes array length: {len(first_ten_million_primes)}\n")
print(f"Gaps between first 10 million primes array length: {len(gaps)}\n")
print(f"Ten millionth prime: {first_ten_million_primes[10000000 - 1]}")

print(f"The count of gaps: \n\n{gap_counts}")

First Ten million primes array length: 10000000

Gaps between first 10 million primes array length: 9999998

Ten millionth prime: 179424673
The count of gaps: 

{2: 738597, 4: 738717, 6: 1297540, 8: 566151, 10: 729808, 12: 920661, 14: 503524, 16: 371677, 18: 667734, 20: 354267, 22: 307230, 24: 453215, 26: 211203, 28: 229177, 30: 398713, 32: 123123, 34: 129043, 36: 206722, 38: 94682, 40: 111546, 42: 159956, 44: 64866, 46: 54931, 48: 93693, 50: 52183, 52: 38800, 54: 64157, 56: 32224, 58: 27985, 60: 55305, 62: 16763, 64: 17374, 66: 30960, 68: 12368, 70: 17475, 72: 17255, 74: 8540, 76: 7253, 78: 13758, 80: 6760, 82: 4791, 84: 9818, 86: 3411, 88: 3454, 90: 7056, 92: 2259, 94: 2058, 96: 3544, 98: 1831, 100: 1923, 102: 2374, 104: 1168, 106: 933, 108: 1634, 110: 941, 112: 711, 114: 1125, 116: 439, 118: 433, 120: 948, 122: 287, 124: 318, 126: 533, 128: 183, 130: 211, 132: 301, 134: 128, 136: 100, 138: 210, 140: 140, 142: 90, 144: 123, 146: 46, 148: 67, 150: 94, 152: 52, 154: 43, 156: 57, 158: 1

# TODO Determine Exact Size of Data to be Compressed


Without actually doing it, imagine creating an ASCII file containing the first ten million primes, represented in decimal, one prime per line. Calculate the size of this file, so you can show an exceptional compression ratio from it (see below).

Using a binary encoding instead of ASCII, each prime requires 32 bits (4 bytes), so the size of a binary file is easily determined.

Using a fixed-width encoding of the gap counts, however, requires knowing how many different gap sizes there are, after which the calculation is straightforward.

In [18]:
from pyprimesieve import primes
import numpy as np
from math import floor
from math import log10

ten_millionth_prime = 179424673
primes = list(primes(ten_millionth_prime+1))

def get_num_digits(n):
    return floor(log10(n)) + 1

ten_million = 10000000 
primes = first_ten_million_primes

# Get the size of the digits, each one has a variable length
total_size_in_digits = sum(map(lambda p: get_num_digits(p), primes))
new_lines = 10 ** 7
megabyte = 2 ** 20

total_size_in_bits = total_size_in_digits * 8

mbs = (total_size_in_digits + new_lines) / megabyte
print(f"File size: {mbs}")

File size: 89.15371894836426


# TODO Use Functional Python


You are encouraged to use the [anytree](https://pypi.org/project/anytree) Python library, which has a nice exporter by way of which you can graphically view trees. (You may recall using this in DM1, and thus know that **anytree** depends on [graphviz](https://graphviz.org), which you also used.)

This library uses the object-oriented features of Python to create and visualize trees. You are encouraged to use the functional features of Python as much as possible, achieving your results not by using some existing third-party libraries for building Huffman Trees and Codes, but writing your own code as cleanly and elegantly as you can.

In [19]:
from math import ceil
from math import log2

print(f"How many gaps: {len(gap_counts)}")
bits_needed = ceil(log2(104)) 
print(f"Bits needed: {bits_needed}")

How many gaps: 104
Bits needed: 7


In [20]:
from pyprimesieve import primes

ten_millionth_prime = 179424673
primes = list(primes(ten_millionth_prime+1))

In [21]:
from collections import Counter
from math import log

def list_the_gaps(n):
    return list(map(lambda i: n[i]-n[i-1],range(2,len(n))))

def get_counted_list_sorted(full_list_of_gaps):
    return Counter(full_list_of_gaps).most_common()

# Returns array of Xth element in array of tuples 
def only_xth_elements_in_array_of_tuples(list_of_tuples, x_element):
    x_element -=1
    grab_x_element = lambda x: x[x_element]
    return [*map(grab_x_element, list_of_tuples)]

get_num_digits = lambda n: len(str(n))

def num_digits_in_elements(list):
    return [*map(get_num_digits, list)]

full_list_of_gaps = list_the_gaps(primes)
sorted_gaps_frequencies = get_counted_list_sorted(full_list_of_gaps)
num_gaps = len(sorted_gaps_frequencies)

gaps = only_xth_elements_in_array_of_tuples(sorted_gaps_frequencies, 1)
freqs = only_xth_elements_in_array_of_tuples(sorted_gaps_frequencies, 2)

num_digits_in_gaps = num_digits_in_elements(gaps)
ascii_sizes_in_bits = [*map(lambda x: (x+1) *8, num_digits_in_gaps)] # 8 bits per character + 1 endline
total_ascii_size = sum([*map(lambda x: (get_num_digits(x) + 1) * 8,primes)]) # add endline * 8 bits


### Used code from CDL

In [22]:
from anytree import Node, RenderTree, PreOrderIter, Walker
from anytree.util import leftsibling, rightsibling
from anytree.exporter.dotexporter import DotExporter
from collections import Counter
from queue import PriorityQueue
from math import ceil, floor, log
from sympy import primerange

def realprimes_up_to(n):
  return list(primerange(4, n))

def get_list_of_gaps(pl):
  gaps_list = list(map(lambda i: pl[i] - pl[i - 1], range(1, len(pl))))
  gaps_list = [2] + gaps_list # [2] for the gap between 3 and 5
  return gaps_list

class GapNode(Node):
  def __lt__(self, other):
    return self.count < other.count

node_counter = 0
def next_node_name(gp):
  global node_counter
  name = 'gn' + str(node_counter) + '_' + str(gp)
  node_counter += 1
  return name

def new_node(gp, ct):
  return GapNode(next_node_name(gp), gap = gp, count = ct)

def new_internal_node(left, right):
  return GapNode(next_node_name(0), children = [left, right],
                 gap = 0, count = left.count + right.count)

def make_huffman_tree(gaps_list):
  gap_dict = Counter(gaps_list)
  q = PriorityQueue()
  for key, val in gap_dict.items():
    q.put(new_node(key, val))

  while q.qsize() > 1:
    left = q.get()
    right = q.get()
    q.put(new_internal_node(left, right))

  return q.get()

def get_codes(root):
  leaves = [node for node in PreOrderIter(root, filter_=lambda n: not n.children)]
  codes = {}
  w = Walker()
  for leaf in leaves:
    path = w.walk(leaf, root)[0]
    code = ''
    for node in path:
      code = ('1' if leftsibling(node) else '0') + code
    codes[leaf.gap] = tuple([code, leaf.count])
  return codes

def compression_ratio(f, v):
  return 100 * (f - v) / f

def get_encoded_size(codes):
  return sum([len(code) * count for gap, (code, count) in codes.items()])

def get_fixed_size(codes):
  num_keys = len(codes)
  num_bits_per_key = ceil(log(num_keys, 2))
  return sum([num_bits_per_key * count for gap, (code, count) in codes.items()])

def report(codes):
  return compression_ratio(get_fixed_size(codes), get_encoded_size(codes))

def test_up_to(primes, upper, results):
  list_of_gaps = get_list_of_gaps(primes[:upper])
  #print(primes[:upper])
  #print(list_of_gaps)
  root = make_huffman_tree(list_of_gaps)
  #print(RenderTree(root))
  DotExporter(root).to_picture(f'gap-tree-{upper:02d}.png')
  codes = get_codes(root)
  print(upper, '-->', get_encoded_size(codes))
  cr = round(report(codes))
  results[upper - 1] = cr # adjust since upper is the number of primes, -1 to make it just odd primes

In [23]:
codes = get_codes(make_huffman_tree(full_list_of_gaps))

In [24]:
total_characters = len(full_list_of_gaps) 
total_unique_characters = len(gaps)
total_bits = sum([*map(lambda x: freqs[x] * len(codes.get(gaps[x])[0]), range(len(freqs)))]) 
fixed_bits_per_character = ceil(log(total_unique_characters + 1, 2)) # TODO fixed bits per character ???
total_fixed_bits = total_characters * fixed_bits_per_character

compression_rat = (total_fixed_bits - total_bits) / total_fixed_bits
#print(f"compression_ratio: {compression_rat}\n") 
print(f"From Fixed:  {compression_rat * 100}")
print(f"from Binary:  {(((9999998 * 32) - total_bits) / (9999998 * 32) * 100):.2f}")
print(f"From ASCII:  {((total_ascii_size - total_bits) / total_ascii_size)* 100 :.2f}")

From Fixed:  36.12516865360516
from Binary:  86.03
From ASCII:  94.02


Your solution should correctly compute the following three compression ratios:

| Ratio       | Value              |
|-------------|--------------------|
| From fixed  | 36.125168653605158 |
| From binary |              86.03 |
| From ASCII  |              94.02 | 


In [25]:
ten_millionth_prime = 179424673
results = {}
test_up_to(primes, ten_millionth_prime+1,results)

179424674 --> 44712402


<img src="gap-tree-179424674.png" alt="image_of_tree_gotten_after_running_code">

# TODO My Report on What I Did and What I Learned

## Fun


I had fun with this assignment. I forgot exactly how you choose the codes for Huffman trees so I had to review that item since it has been a semester since DM1. I explored that until I understood fixed-length encodings and variable-length encodings. I think the code given to us by Br. Neff for the CDL is very helpful. I didn't realize it did all the items relating to the Huffman tree. Also, to see anytree render the picture of the gaps of primes is pretty awesome. 

## New

I relearned Huffman Trees/Encoding I remembered 80% of it. I also learned further how to write functional python. I did my best to understand how to use certain tools I don't always use such as lambda, map, and other items. I try to regularly use items I don't usually so I become more familiar with them and can utilize them when needed.

## Meaningful


I was able to understand encoding and how it can be useful. Collin from class shared a post about Huffman encoding. It talked about how Huffman encoding is used in compressing files. I am curious to find out that when I get a job if I run into encoding problems where I can utilize the knowledge I have gained. Also learning about Huffman trees helps me feel more confident in working with trees which is sometimes more tricky.

## Other

Connections: The main connections I made with something I already knew related to just knowledge of data structures, seeing how a Huffman tree can be formed by using a priority queue or sorting the list by the most frequently occurring items or some priority is interesting. Understanding the structure of why the Huffman tree is encoded in certain ways and the algorithm for assigning codes based on the count of some value is interesting and I enjoyed learning about it because it is a clever and beautiful algorithm.

Collaborator: I worked with Jack Leung we shared our ideas. Compared our answers and conclusions we came to until we felt confident our answers were correct and that we did well.

# TODO What is True?
Click on each warranted checkbox to toggle it to True (or back to False). 

NOTE: *This only works in Colab. If you run it in some other Jupyter notebook client/server environment you may have to change False to True (or vice versa) manually.*

This self-assessment is subject to revision by a grader.

In [26]:
#@markdown ## What is True about what I did?
#@markdown ### I had fun.
cb00 = True #@param {type:'boolean'}
#@markdown ### I learned something new.
cb01 = True #@param {type:'boolean'}
#@markdown ### I achieved something meaningful, or something I can build upon at a later time.
cb02 = True #@param {type:'boolean'}
#@markdown ## What is True about my report?
#@markdown ### I wrote a sufficient number of well-written sentences.
cb03 = True #@param {type:'boolean'}
#@markdown ### My report is free of mechanical infelicities.
cb04 = True #@param {type:'boolean'}
#@markdown ### I used Grammarly (or something better described in my report) to check for MIs.
cb05 = True #@param {type:'boolean'}
#@markdown ### I reported on any connections I found between these problems and something I already know. 
cb06 = True #@param {type:'boolean'}
#@markdown ### I reported who were and what contribution each of my collaborators made.
cb07 = True #@param {type:'boolean'}
#@markdown ## What is True about my calculations?
#@markdown ### I correctly calculated the number of times each gap size occurs. 
cb08 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the number of bits per gap size with a fixed encoding.
cb09 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the total number of bits encoded with the Huffman encoding.
cb10 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the total number of bits encoded with the fixed encoding.
cb11 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from this fixed encoding.
cb12 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the size of the first ten million primes encoded as 32-bit integer binary data.
cb13 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from the binary size.
cb14 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the size of the first ten million primes encoded as ASCII data.
cb15 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from the ASCII size (just the primes, nothing else).
cb16 = True #@param {type:'boolean'}

## DO NOT CHANGE ANYTHING IN THE NEXT CODE CELL!!
### Delete this cell and the following ones before submitting your work.

In [27]:
points_for_what_I_did = [5]*3
points_for_my_report = [8]*5
points_for_my_calculations = [5]*9
points = points_for_what_I_did + points_for_my_report + points_for_my_calculations
# cb is short for checkbox
total = sum(map(lambda n, p: p if eval(f'cb{n:02}') else 0,
                range(len(points)), points))             
total

100

# For graders

In [28]:
#@markdown ---
number_of_MIs_found = 0 #@param {type: 'slider', min: 0, max: 5}
#@markdown ---
